<a href="https://colab.research.google.com/github/kanji709/SimGBS.jl/blob/master/tutorials/SimGBS_Julia_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

<b>SimGBS</b> is a rapid method for <b>simulating Genotyping-By-Sequencing data (GBS)</b>. It can be implemented with any genome of choice. Users can modify different parameters to customise GBS setting, such as the choice of restriction enzyme and sequencing depth. By taking the gene-drop approach, users can also specify the demographic history and define population structure (by supplying a pedigree file). Like real sequencers, SimGBS will output data into FASTQ format.

This notebook follows a [general jupyter notebook template](https://colab.research.google.com/github/ageron/julia_notebooks/blob/master/Julia_Colab_Notebook_Template.ipynb), which has been modified to facilitate implementation of [SimGBS](https://github.com/kanji709/SimGBS.jl) in Julia. To perform proper simulation runs, we strongly recommend running SimGBS in julia directly.

# Setting up the Julia Environment

## <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" />

1. **Make a copy of this notebook**  
   Go to **File → Save a copy in Drive** (you’ll need a Google account).  
   Alternatively, download this notebook and upload it to [Google Colab](https://colab.research.google.com/).

2. **Ensure the runtime type is set to Python 3**  
   Before installing Julia, go to **Runtime → Change runtime type**, and under **Kernel**, select **Python 3**.  
   This is required because the Julia installation process runs from the Python environment.

3. **Install Julia and required packages**  
   Run the setup cell in the next section (click the cell and press **Ctrl + Enter**).  
   This will install Julia and commonly used packages.  
   The process takes a few minutes depending on your connection speed.

4. **Switch to the Julia runtime**  
   Once installation finishes, go again to **Runtime → Change runtime type**,  
   and under **Kernel**, choose **Julia**, then click **Save**.  
   The notebook will reload using the Julia kernel.

---

### Notes

* You must begin in **Python 3** to perform the installation step, then switch to **Julia** afterwards.  
* If your Colab runtime gets reset (for example, after a period of inactivity), repeat steps 2–4.  
* To use a different Julia version, modify the `--channel` flag in the setup cell (for example, `--channel 1.11`).  
* All installed packages and compiled binaries are **temporary** — they will be cleared when the runtime resets.  
* To fully reset the environment, go to **Runtime → Factory reset runtime** and repeat the setup steps.

In [ ]:
%%shell
set -euo pipefail

# --- Configure exactly what you want installed ---
JULIA_VERSION="1.10.5"         # pin a full version, not just "1.10"
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=2
# -------------------------------------------------

want_minor="${JULIA_VERSION%.*}"   # e.g. 1.10
have=""

if command -v julia >/dev/null 2>&1; then
  have=$(julia -e 'print(VERSION)')   # e.g. 1.11.2
  echo "Found existing Julia: $have at $(command -v julia)"
else
  echo "No Julia found on PATH."
fi

# Install/replace if missing or version mismatch
if [ -z "${have:-}" ] || [ "$have" != "$JULIA_VERSION" ]; then
  echo "Installing Julia $JULIA_VERSION ..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$want_minor/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv "$URL" -O /tmp/julia.tar.gz
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz
  echo "Installed Julia at /usr/local/bin/julia -> $(/usr/local/bin/julia -e 'print(VERSION)')"
else
  echo "Desired Julia $JULIA_VERSION already present — skipping install."
fi

# Add GPU package only if a GPU is actually available
if command -v nvidia-smi >/dev/null 2>&1; then
  JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
fi

# Install packages (now definitely using the desired Julia)
for PKG in $JULIA_PACKAGES; do
  echo "Adding Julia package: $PKG"
  julia --color=yes -e 'using Pkg; Pkg.add("'"$PKG"'"); Pkg.precompile()'
done

echo "Installing IJulia kernel named 'julia'..."
julia --color=yes -e 'using IJulia; IJulia.installkernel("julia", env=Dict("JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
KERNEL_DIR=$(julia -e "using IJulia; print(IJulia.kerneldir())")
mv -f "$KERNEL_DIR"/julia* "$KERNEL_DIR"/julia || true

echo "Done. Now switch Colab to Runtime → Change runtime type → Kernel: Julia."

# Checking the Julia Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.10.5
Commit 6f3fdf7b362 (2024-08-27 14:19 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, broadwell)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = auto


In [ ]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  489.435 ms … 612.820 ms  ┊ GC (min … max): 0.00% … 16.76%
 Time  (median):     498.391 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   509.861 ms ±  37.242 ms  ┊ GC (mean ± σ):  2.27% ±  5.24%

  █                                                              
  █▆▁▁▁▁▁▆▆▆▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  489 ms           Histogram: frequency by time          613 ms <

 Memory estimate: 32.00 MiB, allocs estimate: 2.

In [ ]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

No GPU found.


# Install SimGBS

In [ ]:
 import Pkg;Pkg.add("SimGBS")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
Precompiling project...
  ✓ Conda
  1 dependency successfully precompiled in 3 seconds. 198 already precompiled.


In [ ]:
cd("/content")

# Import SimGBS

In [ ]:
using SimGBS

# Download Example Data

In [ ]:
download("https://github.com/kanji709/SimGBS.jl/raw/master/example/ref.fa.gz","ref.fa.gz")
download("https://github.com/kanji709/SimGBS.jl/raw/master/example/GBS_Barcodes.txt","GBS_Barcodes.txt")
readdir()

23-element Vector{String}:
 ".config"
 "ABC12AAXX_1_fastq.txt.gz"
 "GBSCoverage.png"
 "GBSCoverage.txt"
 "GBSFrag.txt"
 "GBSFragSize.png"
 "GBS_Barcodes.txt"
 "RawFrag.txt"
 "RawFragSize.png"
 "keyFile_ABC12AAXX_1.txt"
 "qtlAF.png"
 "qtlGeno.txt"
 "qtlInfo.txt"
 "readDepth.txt"
 "ref.fa.gz"
 "sample_data"
 "shortHap.txt"
 "snpAF.png"
 "snpDepth.txt"
 "snpFragGBS.txt"
 "snpGeno.txt"
 "snpInfo.txt"
 "snpPerTag.png"

# Run SimGBS Example

The code below can be found in [example.jl](https://github.com/kanji709/SimGBS.jl/tree/master/example)

In [ ]:
## parameters
### Step One: Generate GBS Fragments
genofile = "ref.fa.gz"
re = [SimGBS.ApeKI]; # specify the restriction enzyme to be used for virtual digestion
useChr = [1]; # specify either the number of chromosome(s) or a set of chromosome(s) to be used for simulating GBS data
useChrLen = Array{Float64}(undef, 0); # specify the length of chromsome(s) in cM to be simulated, otherwise using the entire chromosome
lower = 65; # lower bound of fragment size selection
upper = 195; # upper bound of fragment size selection
winSize = 1000000 # window size to be used to sample SNP density

### Step Two: Define Population Structure
numFounders = 100; # number of founders in the base population
endSize = 1000; # number of individuals to end up in the changingPopSize step
numGenCha = 20; # number of generations for changingPopSize function
numGenCon = 50; # number of generations for constantPopSize function
numGenFinal = 4; # number of final generations to be used to select individual
numInd = 96; # number of individuals to be simulated
useWeights = Array{Float64}(undef, 0) # weights of each of contributing genetarion in the fianal population composition
usePedigree = false;  # false if you don't use pedigree, otherwise specify the pedigree file to be used
pedFile = "newPed.ped"; # file stores the pedigree (default = "sim.ped")
pedOutput = false; # true if print out pedigree (default = false)

### Step Three: Simulate GBS Process
totalQTL = 100 # total (across all chromosomes) number of QTLs to be simulated
totalSNP = 0 # total (across all chromosomes) number of QTLs to be simulated, set [totalSNP = 0] if sampling based on density
muDensity = -2.0 # location parameter of log-Laplace distribution (for sampling SNP density)
sigmasqDensity = 0.001 # scale parameter of log-Laplace distribution (for sampling SNP density)
winSize = 1000000 # window size to be used to sample SNP density
muAlleleFreq = 0.5 # mean of sampled allele frequency
sigmasqAlleleFreq = 0.0061 # variance of sampled allele frequency
meanDepth = 20.0 # expected sequencing depth
barcodeFile = "GBS_Barcodes.txt" # file contains GBS barcodes
useChr = [1]; # specify either the number of chromosome(s) or a set of chromosome(s) to be used for simulating GBS data

### miscellaneous
plotOutput = true; # true if plots are required
writeOutput = true; # true if outputs in text file are required
onlyOutputGBS = true; # true if only GBS data is required

## Run SimGBS
### Step One: Generate GBS Fragments
@time digestGenome(
    genofile,
    re,
    useChr,
    useChrLen,
    lower,
    upper,
    winSize,
    plotOutput,
    writeOutput,
);

### Step Two: Define Population Structure
@time definePopulation(
    numFounders,
    endSize,
    numGenCha,
    numGenCon,
    numGenFinal,
    numInd,
    useWeights,
    usePedigree,
    pedFile,
    pedOutput,
);

### Step Three: Simulate GBS Process
@time GBS(
    totalQTL,
    totalSNP,
    muDensity,
    sigmasqDensity,
    winSize,
    muAlleleFreq,
    sigmasqAlleleFreq,
    re,
    meanDepth,
    barcodeFile,
    useChr,
    plotOutput,
    writeOutput,
    onlyOutputGBS,
);


INFO: Using chromsome [1] in the simulation!
CHROMOSOME 1: 1800 GBS fragments are generated through virtual digestion using ApeKI.
CHROMOSOME 1: Estimated chromosome length equals to 0.999602 Mb.
CHROMOSOME 1: Subsetting chromosome is not required. Keeping all 1800 available GBS fragments.
INFO: 354 out of 1800 GBS fragments are selected after size-selection with lower and upper thresholds equals to [65, 195].
INFO: Expected sequencing coverage based on 354 selected GBS fragments is approximately 4.56%.
  6.042899 seconds (7.14 M allocations: 512.183 MiB, 2.74% gc time, 96.93% compilation time: 15% of which was recompilation)
CHANGING POP SIZE GEN 10: Done!
CHANGING POP SIZE GEN 20: Done!
CONSTANT POP SIZE GEN 10: Done
CONSTANT POP SIZE GEN 20: Done
CONSTANT POP SIZE GEN 30: Done
CONSTANT POP SIZE GEN 40: Done
INFO: Collecting 22 individual at Gen 47
INFO: Collecting 32 individual at Gen 48
INFO: Collecting 20 individual at Gen 49
INFO: Collecting 22 individual at Gen 50
  5.162813 sec

# Outputs

<b>GBS Fragments</b>

- RawFrag.txt: raw GBS fragments following in slico digestion
- GBSFrag.txt: selected GBS fragments after fragment size-selection
- GBSCoverage.txt: genomic coverage of GBS fragments
- snpFragGBS.txt: GBS fragments that contains SNPs

<b>Variants</b>

- qtlGeno.txt: QTL genotype (number of individual x number of QTL loci)
- snpGeno.txt: SNP genotypes (number of individual x number of QTL loci)
- qtlInfo.txt: information about QTL, including chromosome, position and allele frequency
- snpInfo.txt: information about SNPs, including chromosome, position and allele frequency
- shortHap.txt: short-haplotype created by GBS fragments (i.e., SNPs captured within each GBS fragment)
- readDepth.txt: number of copies per GBS fragment

<b>GBS Data</b>

- keyFileABC12AAXX1.txt: pseudo-information about GBS sample, including flowcell, lane, barcode, sample name, plate, row and column
- ABC12AAXX1fastq.txt.gz: simulated GBS sequences

In [ ]:
readdir() # see outputs

23-element Vector{String}:
 ".config"
 "ABC12AAXX_1_fastq.txt.gz"
 "GBSCoverage.png"
 "GBSCoverage.txt"
 "GBSFrag.txt"
 "GBSFragSize.png"
 "GBS_Barcodes.txt"
 "RawFrag.txt"
 "RawFragSize.png"
 "keyFile_ABC12AAXX_1.txt"
 "qtlAF.png"
 "qtlGeno.txt"
 "qtlInfo.txt"
 "readDepth.txt"
 "ref.fa.gz"
 "sample_data"
 "shortHap.txt"
 "snpAF.png"
 "snpDepth.txt"
 "snpFragGBS.txt"
 "snpGeno.txt"
 "snpInfo.txt"
 "snpPerTag.png"

# Available Functions

## Define Genome

In [ ]:
digestGenome("ref.fa.gz", [SimGBS.ApeKI], [1], Array{Float64}(undef,0), 65 ,195, 1000000, false, true)

INFO: Using chromsome [1] in the simulation!
CHROMOSOME 1: 1800 GBS fragments are generated through virtual digestion using ApeKI.
CHROMOSOME 1: Estimated chromosome length equals to 0.999602 Mb.
CHROMOSOME 1: Subsetting chromosome is not required. Keeping all 1800 available GBS fragments.
INFO: 354 out of 1800 GBS fragments are selected after size-selection with lower and upper thresholds equals to [65, 195].
INFO: Expected sequencing coverage based on 354 selected GBS fragments is approximately 4.56%.


## Define Population

In [ ]:
 definePopulation(100, 500, 20, 100, 4, 96,  Array{Float64}(undef,0), false, "sim.ped", false);

CHANGING POP SIZE GEN 10: Done!
CHANGING POP SIZE GEN 20: Done!
CONSTANT POP SIZE GEN 10: Done
CONSTANT POP SIZE GEN 20: Done
CONSTANT POP SIZE GEN 30: Done
CONSTANT POP SIZE GEN 40: Done
CONSTANT POP SIZE GEN 50: Done
CONSTANT POP SIZE GEN 60: Done
CONSTANT POP SIZE GEN 70: Done
CONSTANT POP SIZE GEN 80: Done
CONSTANT POP SIZE GEN 90: Done
INFO: Collecting 23 individual at Gen 97
INFO: Collecting 26 individual at Gen 98
INFO: Collecting 27 individual at Gen 99
INFO: Collecting 20 individual at Gen 100


## GBS

In [ ]:
GBS(100, 0, -2.0, 0.001, 1000000, 0.5, 0.001, [SimGBS.ApeKI], 20.0, "GBS_Barcodes.txt", [1], false, true, true)

INFO: A total of 100 QTLs are sampled randomly across 1 chromosome(s).
CHROMOSOME 1: 189444 SNPs sampled with average SNP density = 0.135 (window size = 1000000 bp).
CHROMOSOME 1: Found 8487 SNPs on 354 GBS fragments, with an average of 23.97 SNPs per GBS fragment.
[INFO]: a total of 189444 SNPs were sampled.
[INFO]: 8487 SNPs captured by selected GBS SNPs.
CHROMOSOME 1: Filling haplotypes!
CHROMOSOME 1: DONE!
  3.964159 seconds (18.02 M allocations: 697.304 MiB, 2.51% gc time)
[INFO] On Lane 1 of Flowcell ABC12AAXX: Average GBS read depth equals to 20.42.
[INFO] On Lane 1 of Flowcell ABC12AAXX: Generating GBS data for 96 samples.
INFO: A total of 693823 GBS reads generated.


# Overview

For more information, please visit the [documentation](https://kanji709.github.io/SimGBS.jl/dev/) page.



# Citing

Please cite the following if you use SimGBS.jl,

- Hess, A. S., M. K. Hess, K. G. Dodds, J. C. Mcewan, S. M. Clarke, and S. J. Rowe. "A method to simulate low-depth genotyping-by-sequencing data for testing genomic analyses." Proc 11th World Congr Genet Appl to Livest Prod 385 (2018).




# What's Next?

The following tools are recommended for downstream Analyses of GBS data,

[snpGBS](https://github.com/AgResearch/snpGBS): a simple bioinformatics workflow to identify single nucleotide polymorphism (SNP) from Genotyping-by-Sequencing (GBS) data.

[KGD](https://github.com/AgResearch/KGD): R code for the analysis of genotyping-by-sequencing (GBS) data, primarily to construct a genomic relationship matrix for the genotyped individuals.

[GUSLD](https://github.com/AgResearch/GUS-LD): An R package for estimating linkage disequilibrium using low and/or high coverage sequencing data without requiring filtering with respect to read depth.

[SMAP](https://gitlab.com/truttink/smap) a software package that analyzes read mapping distributions and performs haplotype calling to create multi-allelic molecular markers.